In [1]:
import os
import time
import itertools as itr
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
PATH = Path.cwd()
VAR = ['air', 'uwnd', 'vwnd', 'rhum']
VAR_NAME = dict({
    'air': '$T$',
    'uwnd': '$u$',
    'vwnd': '$v$',
    'rhum': '$\\phi$'})
REGRESSORS = list(["Ridge", "RidgeCV", "LinearRegression",
              "GaussianProcessRegressor", "KNeighborsRegressor"])
VAR_DATA_TEST = np.load(PATH.parent/'data/atmos_cond_test.npz')
rs = list(["3", "5", "7"])
p_obs = list(["5", "10", "15"])

In [3]:
for var in VAR:
    data = list()
    for i, exp in enumerate((PATH.parent/"results/exps").glob("*.npz")):
        rg, r, p = exp.name.split(".")[0].split("_")
        results = np.load(exp)
        background = np.load(PATH.parent/f"results/background/{rg}.npz")
        mask = results[var].any(axis=1)
        mse_analysis = ((VAR_DATA_TEST[var][mask][:124] -
                results[var][mask][:124])**2).mean()
        data.append([r, p, rg, np.log10(np.sqrt(mse_analysis))])
    df = pd.DataFrame(
        data=data, columns=["$r$", "$p$", "Regressor", "RMSE"]).pivot(
        index=['$r$', '$p$'], columns="Regressor", values="RMSE")
    df.to_latex(PATH.parent/f"tables/table_rmse_{var}.tex")